In [1]:

import copy
import glob
import os
import re

import conll_transform
from datasets import Dataset, DatasetDict
from stanza.utils.conll import FIELD_TO_IDX, CoNLL
from udapi.block.read.conllu import Conllu as ConlluReader


In [9]:

BASE_PATH = "/Users/ianporada/Research/data/push_to_hub/gum/"
GUM_CONLLU_DIR = BASE_PATH + "gum/dep/"
GUM_CONLL_DIR = BASE_PATH + "gum/coref/gum/conll/"

ONTOGUM_CONLLU_DIR = BASE_PATH + "gum/coref/ontogum/conllu/"
ONTOGUM_CONLL_DIR = BASE_PATH + "gum/coref/ontogum/conll/"

CONLLU_TEXT_COL = FIELD_TO_IDX["text"]
IDX_TO_FIELD = {v:k for k, v in FIELD_TO_IDX.items()}

RAW_DEV_IDS = """
  * GUM_academic_exposure
  * GUM_academic_librarians
  * GUM_bio_byron
  * GUM_bio_emperor
  * GUM_conversation_grounded
  * GUM_conversation_risk
  * GUM_fiction_beast
  * GUM_fiction_lunre
  * GUM_interview_cyclone
  * GUM_interview_gaming
  * GUM_news_homeopathic
  * GUM_news_iodine
  * GUM_reddit_macroeconomics
  * GUM_reddit_pandas
  * GUM_speech_impeachment
  * GUM_speech_inauguration
  * GUM_textbook_governments
  * GUM_textbook_labor
  * GUM_vlog_portland
  * GUM_vlog_radiology
  * GUM_voyage_athens
  * GUM_voyage_coron
  * GUM_whow_joke
  * GUM_whow_overalls
"""

RAW_TEST_IDS = """
  * GUM_academic_discrimination
  * GUM_academic_eegimaa
  * GUM_bio_dvorak
  * GUM_bio_jespersen
  * GUM_conversation_lambada
  * GUM_conversation_retirement
  * GUM_fiction_falling
  * GUM_fiction_teeth
  * GUM_interview_hill
  * GUM_interview_libertarian
  * GUM_news_nasa
  * GUM_news_sensitive
  * GUM_reddit_escape
  * GUM_reddit_monsters
  * GUM_speech_austria
  * GUM_speech_newzealand
  * GUM_textbook_chemistry
  * GUM_textbook_union
  * GUM_vlog_london
  * GUM_vlog_studying
  * GUM_voyage_oakland
  * GUM_voyage_vavau
  * GUM_whow_cactus
  * GUM_whow_mice
"""

In [12]:

def parse_list_of_split_ids(raw_list_of_ids):
    return [x.replace("*", "").strip() for x in raw_list_of_ids.split("\n") if x]


def read_ontogum_conll_file(fname):
    """
    Read ontogum conll file
    each row consists of [global_token_id, token, coref_info]
    """
    lines = []
    with open(fname, "r") as f:
        for line in f:
            line = line.rstrip()
            if not line or line.startswith("#"):
                continue
            row = line.split("\t")
            lines.append(row)

    # hotfix unclosed span and missing token
    basename = os.path.basename(fname)
    if basename == "GUM_voyage_guadeloupe.conll":
        lines[1][-1] = "(1)"
    elif basename == "GUM_news_iodine.conll":
        lines[379][1] = "|"

    return lines


def compute_ontogum_coref_chains(doc_dict, raw_ontogum_lines):
    ontogum_doc_dict = copy.deepcopy(doc_dict)

    doc_as_list = [] # List[List[List]], representing each token in each sentence
    for sentence in ontogum_doc_dict:
        new_sentence = []
        for row in sentence:
            new_row = [row[IDX_TO_FIELD[i]] if IDX_TO_FIELD[i] in row else '_' for i in range(len(IDX_TO_FIELD))]
            new_sentence.append(new_row)
        doc_as_list.append(new_sentence)

    global_token_idx = 0
    for sent in doc_as_list:
        for row in sent:
            raw_ontogum_row = raw_ontogum_lines[global_token_idx]
            global_token_idx += 1
            _, token, coref_info = raw_ontogum_row

            assert row[CONLLU_TEXT_COL] == token, \
                f"Mismatch {row[CONLLU_TEXT_COL]} and {token} at global index: {global_token_idx}"
            row[-1] = coref_info
    ontogum_coref_chains = conll_transform.compute_chains(doc_as_list)
    return ontogum_doc_dict, ontogum_coref_chains


def read_corefud_data(conllu_fname):
    """
    Read sentence metadata and corefud annotations
    """
    docs = ConlluReader(files=conllu_fname, split_docs=True).read_documents()
    assert len(docs) == 1, "GUM should contain one doc per file"
    doc = docs[0]

    sentences = []
    
    for node in doc.nodes_and_empty:

        # assert not node.is_empty(), f"GUM should not have empty nodes: {node} at {node.ord}"
        # nevermind GUM indeed has empty nodes

        # for the first node in the tree, create a new sentence
        if not sentences or node.root.sent_id != sentences[-1]["sent_id"]:
            root = node.root
            speaker = None
            match = re.search("^ speaker = (.+)", root.comment, re.M)
            if match:
                speaker = match.group(1)
            
            sentence = {
                'sent_id': root.sent_id,
                'text': root.text,
                'newpar': root.newpar,
                'newdoc': root.newdoc,
                'global_entity': root.document.meta.get('global.Entity'),
                'comment': root.comment,
                'speaker': speaker,
                'tokens': [],
            }
            sentences.append(sentence)

        coref_mentions = []
        for mention in set(node.coref_mentions):
            coref_mentions.append({
                'span': mention.span,
                'other': mention.other,
                'eid': mention.entity.eid,
                'eid_or_grp': mention.entity.eid_or_grp,
                'etype': mention.entity.etype,
            })

        token = {
            'ord': node.ord,
            'form': node.form,
            'lemma': node.lemma,
            'upos': node.upos,
            'xpos': node.xpos,
            'feats': str(node.feats),
            'head': node.parent.ord if node.parent else None,
            'deprel': node.deprel,
            'misc': str(node.misc),
            'coref_mentions': coref_mentions,
        }
        sentences[-1]['tokens'].append(token)

    coref_entities = []
    for entity in doc.coref_entities:
        coref_mentions = []
        for mention in entity.mentions:
            coref_mentions.append({
                "sent_id": mention.head.root.sent_id,
                'span': mention.span,
                'other': str(mention.other),
                'eid': mention.entity.eid,
                'eid_or_grp': mention.entity.eid_or_grp,
                'etype': mention.entity.etype,
            })
        coref_entities.append(coref_mentions)

    return sentences, coref_entities


# Parse the docs one by one

docs = []
for conllu_fname in glob.glob(os.path.join(GUM_CONLLU_DIR, "*.conllu")):
    doc_id = os.path.basename(conllu_fname).replace(".conllu", "")
    conll_fname = os.path.join(GUM_CONLL_DIR, doc_id + ".conll")
    ontogum_conll_fname = os.path.join(ONTOGUM_CONLL_DIR, doc_id + ".conll")

    # read the conllu file
    doc = CoNLL.conll2doc(conllu_fname)
    doc_dict = doc.to_dict() # List[List[Dict]], representing each token / word in each sentence

    # filter out multi-token rows
    doc_dict = [[row for row in sent if isinstance(row["id"], int)] for sent in doc_dict]

    # read the ontogum annotations
    raw_ontogum_lines = read_ontogum_conll_file(ontogum_conll_fname)
    try:
        ontogum_doc_dict, ontogum_coref_chains = compute_ontogum_coref_chains(doc_dict, raw_ontogum_lines)
    except Exception as e:
        print(f"Error in doc: {doc_id}")
        raise e

    # use udapi to read the corefud annotations from the conllu files
    corefud_sentences, coref_entities = read_corefud_data(conllu_fname)

    # merge doc_dict with corefud_sentences
    merged_sentences = []
    for sent_idx, sent in enumerate(doc_dict):
        corefud_sent = corefud_sentences[sent_idx]

        offset = 0
        for token_idx, token_dict in enumerate(sent):
            conllu_token = token_dict["text"]

            # make sure tokens align between what was read by stanza and udapi
            corefud_row = corefud_sent["tokens"][token_idx + offset]
            while not isinstance(corefud_row["ord"], int):
                offset += 1
                corefud_row = corefud_sent["tokens"][token_idx + offset]
            corefud_token = corefud_row["form"]
            if conllu_token != corefud_token:
                print("Tokens do not match:" \
                      f"{conllu_token} and {corefud_token} in {doc_id} at ({sent_idx}, {token_idx})")
                 

        corefud_sent["conll_rows"] = sent
        merged_sentences.append(corefud_sent)

    doc = {
        "doc_id": doc_id,
        "sentences": merged_sentences,
        "coref_entities": coref_entities,
        "ontogum_sentences": ontogum_doc_dict,
        "ontogum_coref_chains": ontogum_coref_chains,
    }
    docs.append(doc)

/Users/ianporada/Research/envs/venv/lib/python3.12/site-packages/conll_transform/__init__.py:153: UserWarning: Mention (8, 9) duplicated. Ignoring.
  warn(f"Mention {pos} duplicated. Ignoring.")
/Users/ianporada/Research/envs/venv/lib/python3.12/site-packages/conll_transform/__init__.py:153: UserWarning: Mention (4, 23) duplicated. Ignoring.
  warn(f"Mention {pos} duplicated. Ignoring.")
/Users/ianporada/Research/envs/venv/lib/python3.12/site-packages/conll_transform/__init__.py:153: UserWarning: Mention (27, 44) duplicated. Ignoring.
  warn(f"Mention {pos} duplicated. Ignoring.")
/Users/ianporada/Research/envs/venv/lib/python3.12/site-packages/conll_transform/__init__.py:153: UserWarning: Mention (9, 16) duplicated. Ignoring.
  warn(f"Mention {pos} duplicated. Ignoring.")
/Users/ianporada/Research/envs/venv/lib/python3.12/site-packages/conll_transform/__init__.py:153: UserWarning: Mention (1, 9) duplicated. Ignoring.
  warn(f"Mention {pos} duplicated. Ignoring.")
/Users/ianporada/Rese

In [13]:
dev_ids = parse_list_of_split_ids(RAW_DEV_IDS)
test_ids = parse_list_of_split_ids(RAW_TEST_IDS)

train_docs = [doc for doc in docs if doc["doc_id"] not in dev_ids + test_ids]
validation_docs = [doc for doc in docs if doc["doc_id"] in dev_ids]
test_docs = [doc for doc in docs if doc["doc_id"] in test_ids]

In [14]:
print(len(train_docs))

165


In [29]:
for doc in train_docs + validation_docs + test_docs:
    for chain in doc["ontogum_coref_chains"]:
        if len(chain) == 1:
            print(chain)
            print(doc["doc_id"])
            sid = chain[0][0]
            print(doc["ontogum_sentences"][sid])
            break

[(20, 4, 4)]
GUM_reddit_ring
[{'id': 1, 'text': 'Focus', 'lemma': 'focus', 'upos': 'VERB', 'xpos': 'VB', 'feats': 'Mood=Imp|Person=2|VerbForm=Fin', 'head': 0, 'deprel': 'root', 'deps': '0:root', 'misc': 'Discourse=ROOT:29:0'}, {'id': 2, 'text': 'on', 'lemma': 'on', 'upos': 'SCONJ', 'xpos': 'IN', 'head': 3, 'deprel': 'mark', 'deps': '3:mark'}, {'id': 3, 'text': 'singing', 'lemma': 'sing', 'upos': 'VERB', 'xpos': 'VBG', 'feats': 'VerbForm=Ger', 'head': 1, 'deprel': 'advcl', 'deps': '1:advcl:on', 'misc': 'MSeg=sing-ing'}, {'id': 4, 'text': 'whatever', 'lemma': 'whatever', 'upos': 'PRON', 'xpos': 'WP', 'feats': 'PronType=Rel', 'head': 3, 'deprel': 'obj', 'deps': '3:obj', 'misc': 'Entity=(30-abstract-giv:act-cf2-1-coref|MSeg=what-ever'}, {'id': 5, 'text': 'it', 'lemma': 'it', 'upos': 'PRON', 'xpos': 'PRP', 'feats': 'Case=Nom|Gender=Neut|Number=Sing|Person=3|PronType=Prs', 'head': 6, 'deprel': 'nsubj', 'deps': '6:nsubj', 'misc': 'Entity=(30-abstract-giv:act-cf2-1-ana)'}, {'id': 6, 'text': 'i

In [6]:
import datasets

In [ ]:
dataset = datasets.load_dataset

Example: (10) in the following is a singleton in the doc

# sent_id = GUM_reddit_ring-21
# addressee = SkoraSkora
# s_prominence = 1
# s_type = imp
# speaker = YouuCanSingANYTHING
# transition = establishment
# text = Focus on singing whatever it is you want to sing ... not on singing opera!
1	Focus	focus	VERB	VB	Mood=Imp|Person=2|VerbForm=Fin	0	root	0:root	Discourse=ROOT:29:0
2	on	on	SCONJ	IN	_	3	mark	3:mark	_
3	singing	sing	VERB	VBG	VerbForm=Ger	1	advcl	1:advcl:on	MSeg=sing-ing
4	whatever	whatever	PRON	WP	PronType=Rel	3	obj	3:obj	MSeg=what-ever
5	it	it	PRON	PRP	Case=Nom|Gender=Neut|Number=Sing|Person=3|PronType=Prs	6	nsubj	6:nsubj	Entity=(9)
6	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	4	acl:relcl	4:acl:relcl	Entity=(10)
7	you	you	PRON	PRP	Case=Nom|Number=Sing|Person=2|PronType=Prs	8	nsubj	8:nsubj|10:nsubj:xsubj	Entity=(2)
8	want	want	VERB	VBP	Mood=Ind|Number=Sing|Person=2|Tense=Pres|VerbForm=Fin	4	acl:relcl	4:acl:relcl	_
9	to	to	PART	TO	_	10	mark	10:mark	_
10	sing	sing	VERB	VB	VerbForm=Inf	8	xcomp	8:xcomp	_
11	...	…	PUNCT	:	_	14	punct	14:punct	_
12	not	not	PART	RB	Polarity=Neg	14	advmod	14:advmod	Discourse=adversative-antithesis:30->29:0:_
13	on	on	SCONJ	IN	_	14	mark	14:mark	_
14	singing	sing	VERB	VBG	VerbForm=Ger	3	conj	1:advcl:on|3:conj	MSeg=sing-ing
15	opera	opera	NOUN	NN	Number=Sing	14	obj	14:obj	SpaceAfter=No
16	!	!	PUNCT	.	_	1	punct	1:punct	_
